In [1]:
import os
import tensorflow as tf
import json
import pandas as pd
import numpy as np
import random

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
# Folders, named after the file label
labels = ["LEFT_ONE", "LEFT_TWO", "LEFT_THREE",
          "RIGHT_ONE", "RIGHT_TWO", "RIGHT_THREE"]

# Store the dataframes
dataframes = []

# Go through each label
for label in labels:
    # Go through each file in folder
    folder_path = os.listdir(label)

    for file_name in folder_path:
        # Read the file
        df = pd.read_parquet(f"{label}/{file_name}")
        df["sign"] = labels.index(label)
        dataframes.append(df)

    print(f"{label} has been processed!")




LEFT_ONE has been processed!
LEFT_TWO has been processed!
LEFT_THREE has been processed!
RIGHT_ONE has been processed!
RIGHT_TWO has been processed!
RIGHT_THREE has been processed!


In [23]:
dataframes[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, LeftX to RightY
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4 non-null      float64
 1   1       4 non-null      float64
 2   2       4 non-null      float64
 3   3       4 non-null      float64
 4   4       4 non-null      float64
 5   5       4 non-null      float64
 6   6       4 non-null      float64
 7   7       4 non-null      float64
 8   8       4 non-null      float64
 9   9       4 non-null      float64
 10  10      4 non-null      float64
 11  11      4 non-null      float64
 12  12      4 non-null      float64
 13  13      4 non-null      float64
 14  14      4 non-null      float64
 15  15      4 non-null      float64
 16  16      4 non-null      float64
 17  17      4 non-null      float64
 18  18      4 non-null      float64
 19  19      4 non-null      float64
 20  20      4 non-null      float64
 21  sign    4 non-null      int64  
dtypes:

In [24]:
dataframes[0]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,sign
LeftX,0.323437,0.365625,0.400000,0.415625,0.396875,0.346875,0.343750,0.346875,0.343750,0.290625,...,0.381250,0.251563,0.320312,0.343750,0.340625,0.223438,0.279687,0.303125,0.301563,0
RightX,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
LeftY,1.008333,0.989583,0.952083,0.922917,0.910417,0.822917,0.722917,0.650000,0.589583,0.833333,...,0.983333,0.875000,0.906250,0.991667,1.031250,0.935417,0.964583,1.016667,1.037500,0
RightY,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [34]:
# Define empty lists to store features and labels
features_list = []
labels_list = []

for dataframe in dataframes:
    # Extract features (all columns except the last one) and labels (last column)
    features = dataframe.iloc[:, :-1].values
    labels = dataframe.iloc[:, -1].values

    features_list.append(features)
    labels_list.append(labels)

2364


In [26]:
# Stack features and labels into NumPy arrays
features_array = np.vstack(features_list)
labels_array = np.hstack(labels_list)

In [27]:
features_tensor = tf.convert_to_tensor(features_array, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(labels_array, dtype=tf.float32)

In [33]:
import tensorflow as tf
from tensorflow import keras

# Assuming you have features_tensor and labels_tensor ready

# Determine input_shape
input_shape = features_tensor.shape[1]

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_shape,)),  # Input layer
    keras.layers.Dense(128, activation='relu'),        # Hidden layer with 128 neurons and ReLU activation
    keras.layers.Dense(6, activation='softmax')  # Output layer with 'num_classes' and softmax activation
])

# Compile the model
model.compile(optimizer='adam',  # You can use different optimizers like 'adam', 'SGD', etc.
              loss='sparse_categorical_crossentropy',  # For multi-categorical classification
              metrics=['accuracy'])

# Train the model
model.fit(features_tensor, labels_tensor, epochs=100, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(features_tensor, labels_tensor)
print(f'Test accuracy: {test_accuracy}')

# Make predictions
predictions = model.predict(features_tensor)


Epoch 1/100
237/237 [==============================] - 1s 2ms/step - loss: 1.6187 - accuracy: 0.3115 - val_loss: 4.2859 - val_accuracy: 0.0000e+00
Epoch 2/100
237/237 [==============================] - 0s 1ms/step - loss: 1.4407 - accuracy: 0.4067 - val_loss: 5.8410 - val_accuracy: 0.0000e+00
Epoch 3/100
237/237 [==============================] - 0s 1ms/step - loss: 1.3221 - accuracy: 0.4425 - val_loss: 6.7740 - val_accuracy: 0.0037
Epoch 4/100
237/237 [==============================] - 0s 994us/step - loss: 1.2457 - accuracy: 0.4499 - val_loss: 7.4673 - val_accuracy: 0.0428
Epoch 5/100
237/237 [==============================] - 0s 989us/step - loss: 1.1909 - accuracy: 0.4677 - val_loss: 8.0209 - val_accuracy: 0.0201
Epoch 6/100
237/237 [==============================] - 0s 1ms/step - loss: 1.1513 - accuracy: 0.4820 - val_loss: 8.5892 - val_accuracy: 0.0328
Epoch 7/100
237/237 [==============================] - 0s 1ms/step - loss: 1.1208 - accuracy: 0.4952 - val_loss: 9.0825 - val_accu